In [20]:
# Import libraries
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

catalog_lunar_dir = "./data/lunar/data/training/catalogs/apollo12_catalog_GradeA_final.csv"

catalog_lunar = pd.read_csv(catalog_lunar_dir)

catalog_lunar

catalog_lunar.columns
row = catalog_lunar.iloc[6]
arrival_time = datetime.strptime(row["time_abs(%Y-%m-%dT%H:%M:%S.%f)"], '%Y-%m-%dT%H:%M:%S.%f')
arrival_time_relative = row["time_rel(sec)"]
test_filename = row.filename
data_dir = "./data/lunar/data/training/data/S12_GradeA/"
csv_file = f'{data_dir}{test_filename}.csv'
raw_data = pd.read_csv(csv_file)
raw_data

chunk_size = 10000
total_rows = raw_data.shape[0]
total_rows

results_df = pd.DataFrame(columns=["chunk", "label"])
results_df

def extract_features(chunk):
    return [
        chunk['time_rel(sec)'].mean(),
        chunk['time_rel(sec)'].std(),
        chunk['time_rel(sec)'].min(),
        chunk['time_rel(sec)'].max()
    ]

for start in range(0, total_rows, chunk_size):
    end = start + chunk_size

    chunk = raw_data.iloc[start:end]
    # print(chunk)

    data_df = pd.DataFrame(chunk["time_rel(sec)"])
   
    if arrival_time_relative < data_df["time_rel(sec)"].values.max() and arrival_time_relative > data_df["time_rel(sec)"].values.min():
        new_row = pd.DataFrame({"chunk": [chunk], "label": [1]})
        # print(new_row)
        results_df = pd.concat([results_df, new_row], ignore_index=True)
    else:
        new_row = pd.DataFrame({"chunk": [chunk], "label": [0]})
        results_df = pd.concat([results_df, new_row], ignore_index=True)




results_df

label_counts = results_df["label"].value_counts()

count_label_1 = label_counts.get(1, 0)

count_label_1

test_directory = './data/lunar/data/test/data/S15_GradeA/'
row = glob.glob(os.path.join(test_directory, '*.mseed'))

test_filename = row[1]
print(test_filename)

mseed_file = f'{test_filename}'
st = read(mseed_file)
print(st)

tr = st.traces[0].copy()
tr_times = tr.times()
tr_data = tr.data
print(tr_data)

test_df = pd.DataFrame(columns=["chunk"])
total_rows = tr_data.shape[0]


for start in range(0, total_rows, chunk_size):
    end = start + chunk_size

    chunk = raw_data.iloc[start:end]

    data_df = pd.DataFrame(chunk["time_rel(sec)"])

    new_row = pd.DataFrame({"chunk": [chunk]})

    test_df = pd.concat([test_df, new_row], ignore_index=True)



# Assuming 'chunk' column is your input data and 'label' is the target
X_test = np.array([extract_features(chunk) for chunk in test_df['chunk']])
X_train = np.array([extract_features(chunk) for chunk in results_df['chunk']])
y_train = results_df['label'].astype(int)
print(y_train)


# Train Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)
y_pred

# Evaluate model
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Accuracy: {accuracy}')

./data/lunar/data/test/data/S15_GradeA\xa.s15.00.mhz.1973-08-10HR00_evid00126.mseed
1 Trace(s) in Stream:
XA.S15.00.MHZ | 1973-08-10T00:00:00.506000Z - 1973-08-11T00:00:04.883358Z | 6.6 Hz, 572430 samples
[-1.26508535e-15 -1.57797568e-15 -1.70513858e-15 ... -3.02419757e-16
 -3.71501606e-16 -5.48303005e-16]
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
45    0
46    0
47    1
48    0
49    0
50    0
51    0
52    0
53    0
54    0
55    0
56    0
57    0
Name: label, dtype: int64


c:\Users\alvan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])